In [1]:
t1:"[[[[[9,8],1],2],3],4]"
t1

"[[[[[9,8],1],2],3],4]"


In [36]:
/ brackets escaping required https://code.kx.com/q/basics/regex/#matching-special-characters
prepare:{value ssr/[x;("[[]";"[]]";",");"();"]}
pt1:prepare t1
.Q.s1 pt1

"((((9 8;1);2);3);4)"


In [40]:
/ recursively traverse nested lists, flatten result and cut into pairs
traverse:{$[0>type x;(x;y);traverse[;y+1] each x]}
ft1:2 cut (raze/) 0N!traverse[pt1;0]
.Q.s1 ft1 

(((((9 5;8 5);1 4);2 3);3 2);4 1)


"(9 5;8 5;1 4;2 3;3 2;4 1)"


In [41]:
prse:{2 cut (raze/) traverse[prepare x;0]}
.Q.s1 prse t1

"(9 5;8 5;1 4;2 3;3 2;4 1)"


In [42]:
findIdxToExplode:{x[;1]?5} / index of left value of pair to explode
findIdxToExplode ft1

0


In [43]:
removeByIdx:{x (til count x) except y}
removeByIdx[ft1;findIdxToExplode ft1]

8 5
1 4
2 3
3 2
4 1


To explode a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number 0.

In [44]:
explodeAt:{[x;eidx]    
    if[not eidx within (0;-1+count x);'"wrong index to explode",string eidx];
    /show "Exploding ",.Q.s1[x]," at ",string eidx;
    / left value is added to the first regular number to the left of the exploding pair (if any)
    if[eidx>0;x[eidx-1;0]+:x[eidx;0]]; 
    / right value is added to the first regular number to the right of the exploding pair (if any)
    / eidx + 1
    if[eidx<-2+count x;x[eidx+2;0]+:x[eidx+1;0]];
    / replace with 0 and decrease depth by 1
    x[eidx+1]:(0;x[eidx+1;1]-1);
    / remove left value
    removeByIdx[x;eidx]
 };

explode:{eidx:findIdxToExplode x;$[eidx<count x;explodeAt[x;eidx];x]}

In [45]:
prse["[[[[0,9],2],3],4]"]~explode ft1

1b


In [46]:
prse["[7,[6,[5,[7,0]]]]"]~explode prse "[7,[6,[5,[4,[3,2]]]]]"

1b


In [47]:
prse["[[6,[5,[7,0]]],3]"]~explode prse "[[6,[5,[4,[3,2]]]],1]"

1b


In [48]:
prse["[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"]~explode prse "[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]"

1b


In [49]:
prse["[[3,[2,[8,0]]],[9,[5,[7,0]]]]"]~explode prse "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"

1b


In [50]:
findIdxToSplit:{first where x[;0]>=10} / index of left value of pair to explode
findIdxToSplit 0N!prse "[9,[10,2]]"

(9 1;10 2;2 2)


1


To split a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded down, while the right element of the pair should be the regular number divided by two and rounded up. For example, 10 becomes [5,5], 11 becomes [5,6], 12 becomes [6,6], and so on.

In [51]:
splitAt:{[x;sidx]
    if[not sidx within (0;-1+count x);'"wrong index to explode: ",string sidx];
    /show "Splitting ",.Q.s1[x]," at ",string sidx;
    p:(0.5*x[sidx;0];1+x[sidx;1]);
    (floor p;ceiling p);
    :(sidx#x),(floor p;ceiling p),(sidx+1)_x
 };

split:{sidx:findIdxToSplit x;$[null sidx;x;splitAt[x;sidx]]}

In [52]:
.Q.s1 split prse "[[[11,0],10],[10,11]]"

"(5 4;6 4;0 3;10 2;10 2;11 2)"


In [53]:
/ [1,2] + [[3,4],5] becomes [[1,2],[[3,4],5]].
doAdd:{r:x,y;r[;1]+:1;r}

In [54]:
prse["[[1,2],[[3,4],5]]"]~doAdd[prse "[1,2]";prse "[[3,4],5]"]

1b


Here is the process of finding the reduced result of [[[[4,3],4],4],[7,[[8,4],9]]] + [1,1]:

- after addition: [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]
- after explode:  [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
- after explode:  [[[[0,7],4],[15,[0,13]]],[1,1]]
- after split:    [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
- after split:    [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
- after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [55]:
at1:prse"[[[[4,3],4],4],[7,[[8,4],9]]]";
at2:prse"[1,1]";
prse["[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"]~r1:doAdd[at1;at2]

1b


In [56]:
prse["[[[[0,7],4],[7,[[8,4],9]]],[1,1]]"]~r2:explode r1

1b


In [57]:
prse["[[[[0,7],4],[15,[0,13]]],[1,1]]"]~r3:explode r2

1b


In [58]:
prse["[[[[0,7],4],[[7,8],[0,13]]],[1,1]]"]~r4:split r3

1b


In [59]:
prse["[[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]"]~r5:split r4

1b


In [60]:
prse["[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"]~r6:explode r5

1b


In [61]:
prse["[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"]~reduce/[doAdd[at1;at2]]

1b


In [62]:
reduce:{
    eidx:findIdxToExplode x;
    if[eidx<count x;:explodeAt[x;eidx]];
    sidx:findIdxToSplit x;
    if[not null sidx;:splitAt[x;sidx]];
    x
 };

add:{reduce/[doAdd[x;y]]}

In [63]:
prse["[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"]~add[at1;at2]

1b


In [64]:
magnitude:{
    if[0>type x;:x];
    if[0~x[0;1];:x[0;0]];

    midx:x[;1]?max x[;1];
    m:(3*x[midx;0])+2*x[midx+1;0];
    x[midx+1]:(m;x[midx;1]-1);
    removeByIdx[x;midx]
 };

m:{magnitude/[x]}

In [65]:
3488=m[prse "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"]

1b


In [66]:
d:()!()
d[enlist "[[1,2],[[3,4],5]]"]:143
d[enlist "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"]:1384
d[enlist "[[[[1,1],[2,2]],[3,3]],[4,4]]"]:445
d[enlist "[[[[3,0],[5,3]],[4,4]],[5,5]]"]:791
d[enlist "[[[[5,0],[7,4]],[5,5]],[6,6]]"]: 1137
d[enlist "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"]:3488

all {y=0N!m prse x}'[key d;value d]

143
1384
445
791
1137
3488


1b


In [67]:
snums:prse each read0 `:test.txt
snums

(1 1;2 1)
(1 2;2 2;3 1)
(9 1;8 2;7 2)
(1 2;9 2;8 2;5 2)
(1 4;2 4;3 4;4 4;5 4;6 4;7 4;8 4;9 1)
(9 3;3 4;8 4;0 4;9 4;6 3;3 4;7 4;4 4;9 4;3 2)
(1 4;3 4;5 4;3 4;1 4;3 4;8 4;7 4;4 4;9 4;6 4;9 4;8 4;2 4;7 4;3 4)


In [68]:
/ part 1 - accumulate result of sequential addition and calculate magnitude
m add/[first snums;1_snums]

4230


In [69]:
/ all possibe pairs except self
p:c cross c:til count snums
p:p where not p[;0]=p[;1]
3#p

0 1
0 2
0 3


In [70]:
/ part 2 - maximum magnitude of all possible pairs added
max m each add .' snums @ p

4647
